# Model Development and Paramter Tuning

Explain the effects of parameter tuning and how it was performed. If you used a tool to perform automatic tuning , explaing that as well. If it is the case, explain model under or over fitting and the consequences.

## Study Area Description

Model developmnet leverage data from SNOTEL sites located throughout 11 states ranging from the US-Canada border to the US-Mexico border and from the Pacific coast to the eastern slope of the Rocky Mountains to establish the study periphery.
The modeling domain includes all major mountain ranges but contains a greater portion of observations within the Sierra Nevada and the Rocky Mountains, where the water supply is dominated by seasonal snowpack and ongoing efforts by the Airborne Snow Observatory (ASO) provide ample data for data-driven model development.

## Machine Learning Approach

Snow falls different accross the western US, and thus, was decided upon to develop a distributed ML SWE estimation model addressing the unique hydrometeorological variability observed in the modeling doamain.
For example, the western U.S. contains snow climate classifications of coastal, coastal transitional, intermountain, and continental.
The modeling framework addresses the heterogeneity in snow processes through the the division of the study area into 23 regional locations. 
Dividing the model into sub-regions allows for the separation of microclimates to reduce the influence of individual region dynamics on differing regions during model training. 

<img align = 'center' src="./Images/Distribution.jpg" alt = 'drawing' width = '1000'/>


### Data
Machine learning models "learn" the relationships between independent and dependent variables through large amounts of data.
Data sourced for the model consisted of geographic and topographic information from the Copernicus Digital Elevation Model (90-m DEM) and ground measurement data from the NRCS Snow Telemetry and Snow Course program (i.e., SNOTEL), as well as from the California Department of Water Resources California Data Exchange Center (CDEC). 
In total, geographic and weekly SWE observational data from 594 SNOTEL sites and 106 CDEC sites from 2013-2017 are collated.
Weekly observations of the most recent date available at the same locations support near-real-time model inference. 

Model development investigated the use of climatological data from the National Oceanic and Atmospheric Administration's (NOAA) High-Resolution Rapid Refresh dataset (HRRR) and multispectral imagery from the Sentinal-1 remote sensing satellite mission. 
However, the large computational resources required to load and process the data became limiting to the production of expeditious SWE estimates at scale.
The limitations were the result of the large-memory tiled datasets and the need for point-scale architecture required by the ML models to train, test, and predict.
The data processing requirements to convert observations into training datasets are computationally expensive and unfeasible for fast-paced development, even for high-performance computing. 

<img align = 'center' src="Images/Distribution_locations_number.jpg" alt = 'drawing' width = '1000'/>

